### cine21 사이트에서 배우 랭킹 정보 크롤링하기

- http://www.cine21.com/rank/person 접속
- 개발자 도구 실행(F12) 
- NetWork 탭 선택
- Preserve log 선택(체크) 확인
- All 선택 확인
- 설정이 되어있지 않으면 위와 같이 설정하고
- 개발자 도구를 닫은 후 다시 연다
- cine21 화면 중에 
- 집계기간 1개월| 6개월| 1년| 2년 <- 에서 6개월 선택
- Name 밑에 나오는 목록 중에서 content 선택
- 오른쪽 메뉴 중 Headers 선택
- General 의 하위 목록 확인
- Request URL: http://www.cine21.com/rank/person/content
- Request Method: POST
- Status Code: 200 OK
- Remote Address: 115.68.232.4:80
- Referrer Policy: strict-origin-when-cross-origin
- 집계기간 1개월| 6개월| 1년| 2년 <- 에서 2년 선택
- Name 밑에 나오는 목록 중에서 content 선택
- 오른쪽 메뉴 중 Headers 선택
- Request URL: http://www.cine21.com/rank/person/content
- Request Method: POST
- Status Code: 200 OK
- Remote Address: 115.68.232.4:80
- Referrer Policy: strict-origin-when-cross-origin
- 오른쪽 메뉴 중 Payload 선택 및 확인
- section: actor
- period_start: 2020-09
- gender: all
- page: 1

<br>

- 사이트 주소 (URL) : http://www.cine21.com/rank/person
- Request URL: http://www.cine21.com/rank/person/content

- section: actor
- period_start: 2020-09
- gender: all
- page: 1

In [1]:
import pymongo
from bs4 import BeautifulSoup
import requests
import re

In [2]:
'''
1) MongoDB 연결하기
2) Database 생성하기
3) Collection 생성하기
'''
conn = pymongo.MongoClient()
cine = conn.cine
collection_cine = cine.collection_cine

In [3]:
'''
crawling 할 URL 요청하기
'''
url = "http://www.cine21.com/rank/person/content"

'''
비어있는 dictionary 생성하기
section: actor
period_start: 2020-09
gender: all
page: 1
'''
post_data = {}
post_data["section"]="actor"
post_data["period_start"]="2020-09"
post_data["gender"]="all"
post_data["page"]=1
print(post_data)

{'section': 'actor', 'period_start': '2020-09', 'gender': 'all', 'page': 1}


In [4]:
'''
post 방식으로 request 보내기
'''
resp = requests.post(url, post_data)
print(resp)

<Response [200]>


In [5]:
resp.content

b'\t\t\t<ul class="people_list">\r\n\t\t\t\t<li class="people_li">\r\n\t\t\t\t\t<a href="/db/person/info/?person_id=15222"><img src="https://image.cine21.com/resize/cine21/still/2017/1207/14_33_48__5a28d2bcb1647[X145,145].jpg" alt="" class="people_thumb" target="_blank" /></a>\r\n\t\t\t\t\t<div class="name"><a href="/db/person/info/?person_id=15222">\xeb\xb0\x95\xed\x95\xb4\xec\x9d\xbc(5\xed\x8e\xb8)</a></div>\r\n\t\t\t\t\t<ul class="num_info">\r\n\t\t\t\t\t\t<li><span class="tit">\xed\x9d\xa5\xed\x96\x89\xec\xa7\x80\xec\x88\x98</span><strong>23,925</strong></li>\r\n\t\t\t\t\t\t<!--\r\n\t\t\t\t\t\t<li><a href="#" class="btn_graph"><span class="ico"></span><span>\xed\x9d\xa5\xed\x96\x89\xec\x84\xb1\xec\xa0\x81<br />\xea\xb7\xb8\xeb\x9e\x98\xed\x94\x84\xeb\xa1\x9c \xeb\xb3\xb4\xea\xb8\xb0</span></a></li>\r\n\t\t\t\t\t\t-->\r\n\t\t\t\t\t</ul>\r\n\t\t\t\t\t<!-- \xec\x98\x81\xed\x99\x94\xed\x8f\xac\xec\x8a\xa4\xed\x84\xb0\xeb\x8a\x94 \xec\xb5\x9c\xeb\x8c\x80 5\xea\xb0\x9c\xea\xb9\x8c\xec\xa

### 응답(Response) 받은 문서를 parsing 해서 원하는 data 추출하기

In [6]:
soup = BeautifulSoup(resp.content, "html.parser")

In [7]:
soup

 <ul class="people_list">
<li class="people_li">
<a href="/db/person/info/?person_id=15222"><img alt="" class="people_thumb" src="https://image.cine21.com/resize/cine21/still/2017/1207/14_33_48__5a28d2bcb1647[X145,145].jpg" target="_blank"/></a>
<div class="name"><a href="/db/person/info/?person_id=15222">박해일(5편)</a></div>
<ul class="num_info">
<li><span class="tit">흥행지수</span><strong>23,925</strong></li>
<!--
						<li><a href="#" class="btn_graph"><span class="ico"></span><span>흥행성적<br />그래프로 보기</span></a></li>
						-->
</ul>
<!-- 영화포스터는 최대 5개까지만 -->
<ul class="mov_list">
<li>
<a href="/movie/info/?movie_id=56294">
<img alt="" class="thumb" src="https://image.cine21.com/resize/cine21/poster/2022/0727/16_21_10__62e0e76663483[X85,120].jpg" target="_blank"/>
<span>한산: 용의 출현</span>
</a>
</li>
<li>
<a href="/movie/info/?movie_id=56942">
<img alt="" class="thumb" src="https://image.cine21.com/resize/cine21/poster/2022/0610/09_59_51__62a2978774d80[X85,120].jpg" target="_blank"/>
<span>헤어질 

In [8]:
actors = soup.select("li.people_li div.name")

for actor in actors:
    print(actor)

<div class="name"><a href="/db/person/info/?person_id=15222">박해일(5편)</a></div>
<div class="name"><a href="/db/person/info/?person_id=35760">박지환(7편)</a></div>
<div class="name"><a href="/db/person/info/?person_id=104968">손석구(3편)</a></div>
<div class="name"><a href="/db/person/info/?person_id=20745">현빈(4편)</a></div>
<div class="name"><a href="/db/person/info/?person_id=102264">최귀화(6편)</a></div>
<div class="name"><a href="/db/person/info/?person_id=98611">조우진(9편)</a></div>
<div class="name"><a href="/db/person/info/?person_id=91462">허동원(4편)</a></div>


In [9]:
'''
actor.text : content 만 추출함
'''
for actor in actors:
    print(actor.text)

박해일(5편)
박지환(7편)
손석구(3편)
현빈(4편)
최귀화(6편)
조우진(9편)
허동원(4편)


In [10]:
print(type(actor.text))

<class 'str'>


In [11]:
'''
배우이름만 추출함 : 문자열의 split() 메소드 호출
'''
for actor in actors:
    print((actor.text).split("("))

['박해일', '5편)']
['박지환', '7편)']
['손석구', '3편)']
['현빈', '4편)']
['최귀화', '6편)']
['조우진', '9편)']
['허동원', '4편)']


In [12]:
for actor in actors:
    print((actor.text).split("(")[0])

박해일
박지환
손석구
현빈
최귀화
조우진
허동원


In [13]:
for actor in actors:
    print(actor.select_one("a"))

<a href="/db/person/info/?person_id=15222">박해일(5편)</a>
<a href="/db/person/info/?person_id=35760">박지환(7편)</a>
<a href="/db/person/info/?person_id=104968">손석구(3편)</a>
<a href="/db/person/info/?person_id=20745">현빈(4편)</a>
<a href="/db/person/info/?person_id=102264">최귀화(6편)</a>
<a href="/db/person/info/?person_id=98611">조우진(9편)</a>
<a href="/db/person/info/?person_id=91462">허동원(4편)</a>


In [14]:
'''
http://www.cine21.com/db/person/info/?person_id=15222
'''

'\nhttp://www.cine21.com/db/person/info/?person_id=15222\n'

In [15]:
for actor in actors:
    print(actor.select_one("a").attrs["href"])

/db/person/info/?person_id=15222
/db/person/info/?person_id=35760
/db/person/info/?person_id=104968
/db/person/info/?person_id=20745
/db/person/info/?person_id=102264
/db/person/info/?person_id=98611
/db/person/info/?person_id=91462


In [16]:
for actor in actors:
    print("http://www.cine21.com" + actor.select_one("a").attrs["href"])

http://www.cine21.com/db/person/info/?person_id=15222
http://www.cine21.com/db/person/info/?person_id=35760
http://www.cine21.com/db/person/info/?person_id=104968
http://www.cine21.com/db/person/info/?person_id=20745
http://www.cine21.com/db/person/info/?person_id=102264
http://www.cine21.com/db/person/info/?person_id=98611
http://www.cine21.com/db/person/info/?person_id=91462


### 정규표현식 : Regualr Expression
```html

  https://regexr.com/

  <li><span class="tit">직업</span>배우</li>

  - .  : \n(줄바꿈)문자를 제외한 모든 문자 하나를 의미함
  - *  : 앞 문자가 반복되는 pattern
  - .* : 문자가 반복되는 pattern


```

In [17]:
'''
get 방식으로 request 보내기
  ㄴ requests 모듈에 있는 get() 함수를 호출함
  
  ul.default_info
    <ul class="default_info">
        <li><span class="tit">직업</span>배우</li>
        <li><span class="tit">생년월일</span>1977-01-26</li>

        <li><span class="tit">성별</span>남</li>


        <li><span class="tit">신장/체중</span>177cm, 65kg</li>
        <li><span class="tit">학교</span>남서울대학교 영어학 (중퇴)</li>
        <li><span class="tit">취미</span>영화감상, 음악감상, 컴퓨터게임</li>
        <li><span class="tit">특기</span>노래, 기타연주, 탁구, 야구</li>

        <li><span class="tit">소속사</span>휴메인엔터테인먼트</li>
    </ul>
    
'''

# 비어있는 list 생성
actors_info_list = []

for actor in actors:
    actor_link = "http://www.cine21.com" + actor.select_one("a").attrs["href"]
    resp_actor = requests.get(actor_link)
    soup_actor = BeautifulSoup(resp_actor.content, "html.parser")
    
    default_info = soup_actor.select_one("ul.default_info")
    details_actor = default_info.select("li")
    
    # 비어있는 dictionary 생성
    actors_info_dict = {}
    
    for actor_item in details_actor:
        actor_item_key = actor_item.select_one("span.tit").text
        actor_item_value = re.sub("<span.*?>.*?</span>","",str(actor_item))
        actor_item_value = re.sub("<.*?>","",actor_item_value)
        actors_info_dict[actor_item_key] = actor_item_value
    
    actors_info_list.append(actors_info_dict)
    
actors_info_list
    

[{'직업': '배우',
  '생년월일': '1977-01-26',
  '성별': '남',
  '신장/체중': '177cm, 65kg',
  '학교': '남서울대학교 영어학 (중퇴)',
  '취미': '영화감상, 음악감상, 컴퓨터게임',
  '특기': '노래, 기타연주, 탁구, 야구',
  '소속사': '휴메인엔터테인먼트'},
 {'직업': '배우', '생년월일': '1980-09-05', '성별': '남'},
 {'직업': '배우', '생년월일': '1983-00-00', '성별': '남'},
 {'원어명': '玄彬',
  '다른 이름': '김태평',
  '직업': '배우',
  '생년월일': '1982-09-25',
  '성별': '남',
  '홈페이지': '\nhttp://www.vastenm.com/\n',
  '신장/체중': '184cm, 74kg',
  '학교': '중앙대학교 연극영화 졸업, 중앙대학교 예술 대학원 공연영상학과 재학 중',
  '취미': '수영, 농구, 야구',
  '특기': '스노우보드, 사진',
  '소속사': 'VAST엔터테인먼트'},
 {'직업': '배우', '생년월일': '1978-03-03', '성별': '남', '신장/체중': '181cm, 72kg'},
 {'다른 이름': '조신제', '직업': '배우', '생년월일': '1979-01-16', '성별': '남'},
 {'직업': '배우', '생년월일': '1980-06-09', '성별': '남'}]

### 각 배우들의 흥행지수 / 출연 영화

In [18]:
actors = soup.select("li.people_li div.name")
hits = soup.select("ul.num_info > li > strong")
movies = soup.select("ul.mov_list")

for actor in actors:
    print(actor)
    
for hit in hits:
    print(hit)
    
print('hits[0] :',hits[0]) 
print('-' * 60)


<div class="name"><a href="/db/person/info/?person_id=15222">박해일(5편)</a></div>
<div class="name"><a href="/db/person/info/?person_id=35760">박지환(7편)</a></div>
<div class="name"><a href="/db/person/info/?person_id=104968">손석구(3편)</a></div>
<div class="name"><a href="/db/person/info/?person_id=20745">현빈(4편)</a></div>
<div class="name"><a href="/db/person/info/?person_id=102264">최귀화(6편)</a></div>
<div class="name"><a href="/db/person/info/?person_id=98611">조우진(9편)</a></div>
<div class="name"><a href="/db/person/info/?person_id=91462">허동원(4편)</a></div>
<strong>23,925</strong>
<strong>22,069</strong>
<strong>21,422</strong>
<strong>18,211</strong>
<strong>18,131</strong>
<strong>16,499</strong>
<strong>16,039</strong>
hits[0] : <strong>23,925</strong>
------------------------------------------------------------


In [19]:

for idx, actor in enumerate(actors):
    print("배우이름 :",(actor.text).split("(")[0])
    print("흥행지수 :",int(hits[idx].text.replace(",","")))
    movie_titles = movies[idx].select("li a span")
    
    movie_titles_list = []    
    for movie_title in movie_titles:
        movie_titles_list.append(movie_title.text)
        
    print("출연영화 :",movie_titles_list)
    print("-" * 80)

배우이름 : 박해일
흥행지수 : 23925
출연영화 : ['한산: 용의 출현', '헤어질 결심', '와이키키 브라더스', '제보자', '후아유']
--------------------------------------------------------------------------------
배우이름 : 박지환
흥행지수 : 22069
출연영화 : ['유체이탈자', '한산: 용의 출현', '해적: 도깨비 깃발', '지푸라기라도 잡고 싶은 짐승들', '범죄도시2', '봉오동 전투']
--------------------------------------------------------------------------------
배우이름 : 손석구
흥행지수 : 21422
출연영화 : ['범죄도시2', '연애 빠진 로맨스', '뺑반']
--------------------------------------------------------------------------------
배우이름 : 현빈
흥행지수 : 18211
출연영화 : ['공조2: 인터내셔날', '만추', '협상', '공조']
--------------------------------------------------------------------------------
배우이름 : 최귀화
흥행지수 : 18131
출연영화 : ['부기나이트', '기방도령', '범죄도시2', '원더풀 고스트', '택시운전사', '조작된 도시']
--------------------------------------------------------------------------------
배우이름 : 조우진
흥행지수 : 16499
출연영화 : ['외계+인 1부', '발신제한', '킹메이커', '도굴', '서복', '국가부도의 날']
--------------------------------------------------------------------------------
배우이름 : 허동원
흥행지수 : 16039
출연영화 : [

### 수집한 data 로 dictionary 생성하기
-  흥행지수 + 출연영화 + 배우상세정보

In [20]:
actors_info_list = []

actors = soup.select("li.people_li div.name")
hits = soup.select("ul.num_info > li > strong")
movies = soup.select("ul.mov_list")

for idx, actor in enumerate(actors):
    actor_name = (actor.text).split("(")[0]
    actor_hits  = int(hits[idx].text.replace(",", ""))
    movie_titles = movies[idx].select("li a span")
    
    movie_title_list = []
    for movie_title in movie_titles:
        movie_title_list.append(movie_title.text)
        
    actors_info_dict = {}      
    actors_info_dict["배우이름"] = actor_name 
    actors_info_dict["흥행지수"] = actor_hits
    actors_info_dict["출연영화"] = movie_title_list
    
    actor_link = "http://www.cine21.com" + actor.select_one("a").attrs["href"]
    actor_resp = requests.get(actor_link)
    actor_soup = BeautifulSoup(actor_resp.content, "html.parser")    
    actor_details = default_info.select("li")
    
    for actor_item in actor_details:
        actor_item_key = actor_item.select_one("span.tit").text
        actor_item_value = re.sub("<sapn.*?>.*?</span>", "", str(actor_item))
        actor_item_value = re.sub("<.*?>", "", actor_item_value)
        
        actors_info_dict[actor_item_key] = actor_item_value
        
    actors_info_list.append(actors_info_dict)    
    
actors_info_list    

[{'배우이름': '박해일',
  '흥행지수': 23925,
  '출연영화': ['한산: 용의 출현', '헤어질 결심', '와이키키 브라더스', '제보자', '후아유'],
  '직업': '직업배우',
  '생년월일': '생년월일1980-06-09',
  '성별': '성별남'},
 {'배우이름': '박지환',
  '흥행지수': 22069,
  '출연영화': ['유체이탈자',
   '한산: 용의 출현',
   '해적: 도깨비 깃발',
   '지푸라기라도 잡고 싶은 짐승들',
   '범죄도시2',
   '봉오동 전투'],
  '직업': '직업배우',
  '생년월일': '생년월일1980-06-09',
  '성별': '성별남'},
 {'배우이름': '손석구',
  '흥행지수': 21422,
  '출연영화': ['범죄도시2', '연애 빠진 로맨스', '뺑반'],
  '직업': '직업배우',
  '생년월일': '생년월일1980-06-09',
  '성별': '성별남'},
 {'배우이름': '현빈',
  '흥행지수': 18211,
  '출연영화': ['공조2: 인터내셔날', '만추', '협상', '공조'],
  '직업': '직업배우',
  '생년월일': '생년월일1980-06-09',
  '성별': '성별남'},
 {'배우이름': '최귀화',
  '흥행지수': 18131,
  '출연영화': ['부기나이트', '기방도령', '범죄도시2', '원더풀 고스트', '택시운전사', '조작된 도시'],
  '직업': '직업배우',
  '생년월일': '생년월일1980-06-09',
  '성별': '성별남'},
 {'배우이름': '조우진',
  '흥행지수': 16499,
  '출연영화': ['외계+인 1부', '발신제한', '킹메이커', '도굴', '서복', '국가부도의 날'],
  '직업': '직업배우',
  '생년월일': '생년월일1980-06-09',
  '성별': '성별남'},
 {'배우이름': '허동원',
  '흥행지수': 16039,
  '출연영화': ['지푸라기라도 잡고 싶은 